# Trabajo Text mining en Twitter

El trabajo básico no presencial de "Text Mining en Twitter" consistirá en desarrollar un pequeño sistema de Sentiment Analysis siguiendo los siguientes pasos:
 
0) obtener una cuenta de desarrollo en Twitter (apps.twitter.com), registrar una aplicación y las claves correspondiente.

1) descargar los tweets de entrenamiento a partir del id (TASS_training_polarity.txt).

2) utilizar algún método de machine learning para construir un clasificador.

3) descargar los tweets de test a partir del id (TASS_test_ids.txt).

4) etiquetar los tweets de test utilizando el modelo aprendido en 2).

5) subir a la actividad de PoliformaT un fichero de texto con el test etiquetado (identificador \t polaridad) y un fichero ".zip" con todo el código desarrollado.  

6) para mejorar los modelos se puede utilizar un diccionario de polaridad (ElhPolar_esV1.lex )

El formato del fichero de test debe ser igual que el de train (TASS_training_polarity.txt) es decir:

identificador1 (tabulador) polaridad1
identificador2 (tabulador) polaridad2
identificador3 (tabulador) polaridad3

Se debe subir todo el código y el fichero del test etiquetado.

## Import twitter credentials

In [2]:
import tweepy
import json

from twitter_secret import *

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

API = tweepy.API(auth)

print (API)

## Read text TASS

In [3]:
import csv

text_file = open('TASS_training_polarity.txt', 'r')
#text_file = open('TASS_test_ids.txt', 'r')

list = {}
idList = []

for row in csv.reader(text_file, delimiter='\t'):
    id = row[0]
    if(len(row) > 1):
        score = row[1]
    else:
        score = None
    
    json = {id: {'score': score}} if score else {id: {}}
        
    idList.append(id)
    list.update(json)


## Read tweets

Twitter API allows to retrieve tweets 100 by 100, so page the ids in order to fit their contraints

In [6]:
import math

numPages = int(math.ceil(len(idList)/100))

for i in range(0, numPages):
    tweetPage = API.statuses_lookup(idList[i*100:i*100+99])
    for idx, tweet in enumerate(tweetPage):
        if(list.get(tweet.id_str).get('score')):
            json = { tweet.id_str: {'score': list.get(tweet.id_str).get('score'), 'text': tweet.text, 'user': tweet.user.screen_name }  }
        else:
            json = { tweet.id_str: {'text': tweet.text, 'user': tweet.user.screen_name } }
        
        list.update(json)


## Transform to csv rows

In [8]:
dataRows = []
for key, value in list.items():
    row = []
    if('text' in value):
        row.append(key)
        row.append(value['text'].replace('\n', ' '))
        row.append(value['score'])
        row.append(value['user'])
        dataRows.append(row)
    
# leemos el % de tweets que han sido descargados correctamente
print('Descargados correctamente: {}/{}'.format(len(dataRows), len(list.items())))

Descargados correctamente: 6844/7219


## Save to csv, not to load them all again

In [9]:
# -*- coding: utf-8 -*-
import csv

def save_tweets(filename, tweets):
    with open(filename, 'w', encoding='utf-8') as fh:
        for tweet in tweets:
            line = '\t'.join(tweet)
            fh.write(line + '\n')
    fh.close()

save_tweets('tweets.csv', dataRows)

print('Saved')

Saved
